In [1]:
from qiskit_ibm_runtime.fake_provider import FakeTorino

from qiskit import QuantumCircuit

backend = FakeTorino()

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)

import time

Bell State

In [10]:
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

# Get ISA circuit
pm_bell = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_qc = pm_bell.run(qc)

# Get gate counts
gate_counts = isa_qc.count_ops()
print(gate_counts)

OrderedDict({'rz': 6, 'sx': 3, 'measure': 2, 'cz': 1, 'barrier': 1})


Ising with transverse and longitudinal fields Hamiltonian, nearest neighbor ring

In [3]:
num_qubits = 50
qc=QuantumCircuit(num_qubits)
for i in range(num_qubits):    
    j = (i + 1) % num_qubits
    
    qc.rx(0.1, i)
    qc.rz(0.2, i)
    
    qc.rx(0.1, j)
    qc.rz(0.2, j)
    
    qc.cx(i, j)
    qc.rz(0.1, j)
    qc.cx(i, j)

# Get ISA circuit
t1 = time.time()
isa_qc = pm.run(qc)
t2 = time.time()
print("Time taken: ", t2-t1)

# Get gate counts
gate_counts = isa_qc.count_ops()
print(gate_counts)
print("Number of 2-qubit gates: ", gate_counts.get("cz", 0) + gate_counts.get("cx", 0)) 
print("Number of 1-qubit gates: ", gate_counts.get("sx", 0) + gate_counts.get("rz", 0) + gate_counts.get("x", 0))

Time taken:  21.209616899490356
OrderedDict({'sx': 765, 'rz': 403, 'cz': 331, 'x': 43})
Number of 2-qubit gates:  331
Number of 1-qubit gates:  1211


Ising with transverse and longitudinal fields Hamiltonian, all-to-all

In [4]:
num_qubits = 10
qc=QuantumCircuit(num_qubits)
for i in range(num_qubits):
    for j in range(i+1, i+num_qubits):
        j = j % num_qubits

        qc.rx(0.1, i)
        qc.rz(0.2, i)
        
        qc.rx(0.1, j)
        qc.rz(0.2, j)
        
        qc.cx(i, j)
        qc.rz(0.1, j)
        qc.cx(i, j)

# Get ISA circuit
t1 = time.time()
isa_qc = pm.run(qc)
t2 = time.time()
print("Time taken: ", t2-t1)

# Get gate counts
gate_counts = isa_qc.count_ops()
print(gate_counts)
print("Number of 2-qubit gates: ", gate_counts.get("cz", 0) + gate_counts.get("cx", 0)) 
print("Number of 1-qubit gates: ", gate_counts.get("sx", 0) + gate_counts.get("rz", 0) + gate_counts.get("x", 0))
        

Time taken:  0.660789966583252
OrderedDict({'sx': 896, 'rz': 753, 'cz': 369, 'x': 101})
Number of 2-qubit gates:  369
Number of 1-qubit gates:  1750


QCNN

In [5]:
num_qubits = 50
num_layers = 5

qc=QuantumCircuit(num_qubits)
i_conv=0
for i_layer in range(num_layers):
    for i_sub_layer in [0 , 2**i_layer]:            
        for i_q1 in range(i_sub_layer, num_qubits, 2**(i_layer+1)):
            i_q2=2**i_layer+i_q1
            if i_q2<num_qubits:
                qc.rxx(0.1, i_q1, i_q2)
                i_conv+=1

# Get ISA circuit
t1 = time.time()
isa_qc = pm.run(qc)
t2 = time.time()
print("Time taken: ", t2-t1)

# Get gate counts
gate_counts = isa_qc.count_ops()
print(gate_counts)
print("Number of 2-qubit gates: ", gate_counts.get("cz", 0) + gate_counts.get("cx", 0)) 
print("Number of 1-qubit gates: ", gate_counts.get("sx", 0) + gate_counts.get("rz", 0) + gate_counts.get("x", 0))

Time taken:  0.9238553047180176
OrderedDict({'sx': 924, 'rz': 603, 'cz': 431, 'x': 65})
Number of 2-qubit gates:  431
Number of 1-qubit gates:  1592
